## Gerekli Kütüphanelerin Yüklenmesi ve Datanın Okunması

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score,mean_squared_error,r2_score,roc_auc_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
import warnings
warnings.filterwarnings("ignore",category =DeprecationWarning)
warnings.filterwarnings("ignore",category =FutureWarning)

In [3]:
df = pd.read_csv("diabetes.csv")

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


## Model Kurma - Tahmin

In [7]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [11]:
y = df["Outcome"]
X= df.drop(["Outcome"],axis=1)

In [12]:
lojistic_model = LogisticRegression(solver="liblinear").fit(X,y)

In [13]:
lojistic_model.intercept_

array([-5.89159518])

In [14]:
lojistic_model.predict(X[:10])

array([1, 0, 1, 0, 1, 0, 0, 1, 1, 0], dtype=int64)

In [15]:
y[:10]

0    1
1    0
2    1
3    0
4    1
5    0
6    1
7    0
8    1
9    1
Name: Outcome, dtype: int64

In [16]:
y_pred = lojistic_model.predict(X)

In [17]:
confusion_matrix(y,y_pred)

array([[448,  52],
       [121, 147]], dtype=int64)

In [19]:
accuracy_score(y,y_pred)

0.7747395833333334

In [20]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       500
           1       0.74      0.55      0.63       268

    accuracy                           0.77       768
   macro avg       0.76      0.72      0.73       768
weighted avg       0.77      0.77      0.77       768



## Model Tuning

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)

In [25]:
lojistic_model = LogisticRegression(solver="liblinear").fit(X_train,y_train)

In [26]:
y_pred = lojistic_model.predict(X_test)

In [27]:
print(accuracy_score(y_test,y_pred))

0.7532467532467533


In [29]:
cross_val_score(lojistic_model,X_test,y_test,cv=10).mean()

0.7704710144927536

## Destek Vektör Makineleri - Model Eğitimi

In [30]:
from sklearn.svm import SVC

In [31]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [32]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)

In [33]:
svc_model = SVC(kernel="linear").fit(X_train,y_train)

In [34]:
svc_model

SVC(kernel='linear')

In [35]:
y_pred = svc_model.predict(X_test)

In [37]:
accuracy_score(y_test,y_pred)

0.7445887445887446

## Model Tuning

In [38]:
svm_params = {"C":np.arange(1,10),"kernel":["linear","rbf"]}

In [39]:
svm= SVC()

In [40]:
svm_cv_model = GridSearchCV(svm,svm_params,cv=2,n_jobs=-1,verbose=2).fit(X_train,y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.5min finished


In [47]:
svm_cv_model.best_score_

0.7746768018642846